In [ ]:
from os import path
from pprint import pprint
from corputil import FileCorpus
from gensim.corpora import Dictionary
from gensim.models import LdaMulticore
from gensim.models.phrases import Phrases

In [ ]:
num_topics = 20
chunksize, iterations, passes = 500, 500, 20
labels = ['KW44', 'KW45', 'KW46', 'KW47', 'KW48', 'KW49']
files = [path.join('data', 'Corpus_{}.txt').format(label) for label in labels]
output_model = [path.join('models', 'lda', '{}.lda').format(label) for label in labels]
output_dict = [path.join('models', 'lda', '{}.dict').format(label) for label in labels]
output_mm = [path.join('models', 'lda', '{}.mm').format(label) for label in labels]
corpora = [FileCorpus(file) for file in files]

def train_lda(train):
    corpus = [trigram[bigram[doc]] for doc in train]

    dictionary = Dictionary(corpus)
    dictionary.filter_extremes()
    dictionary.compactify()

    mmCorpus = [dictionary.doc2bow(doc) for doc in corpus]
    lda = LdaMulticore(mmCorpus, id2word=dictionary, chunksize=chunksize, batch=True,
                       num_topics=num_topics, workers=2, passes=passes, iterations=iterations)
    return (dictionary, mmCorpus, lda)

In [ ]:
bigram = Phrases.load(path.join('models', 'phrase', 'Bigram.phrase'))
trigram = Phrases.load(path.join('models', 'phrase', 'Trigram.phrase'))

for i, corpus in enumerate(corpora):
    dictionary, corpus, model = train_lda(corpus.doc_token(stopwords='german'))
    model.save(output_model[i])
    dictionary.save(output_dict[i])
    corpus.save(output_mm[i])